In [8]:
import tkinter as tk
%run ./base_exercises.ipynb

# Create a function to display exercise details in a window
def display_exercise_details(exercise):
    # Create a Tkinter window
    window = tk.Tk()
    window.title("Exercise Details")

    # Create labels to display exercise details
    tk.Label(window, text="Name:").grid(row=0, column=0, sticky="w")
    tk.Label(window, text=exercise.name).grid(row=0, column=1, sticky="w")

    tk.Label(window, text="Description:").grid(row=1, column=0, sticky="w")
    tk.Label(window, text=exercise.description).grid(row=1, column=1, sticky="w")

    tk.Label(window, text="Difficulty:").grid(row=2, column=0, sticky="w")
    tk.Label(window, text=exercise.difficulty).grid(row=2, column=1, sticky="w")

    tk.Label(window, text="Instructions:").grid(row=3, column=0, sticky="w")
    instructions_text = tk.Text(window, height=6, width=40)
    instructions_text.insert(tk.END, exercise.instructions)
    instructions_text.grid(row=3, column=1, sticky="w")

    # Run the Tkinter event loop
    window.mainloop()

# Example usage
push_up = BaseExercise(
    "Push-Ups",
    "Upper Body Strengthening Movement",
    30,
    "medium",
    ["chest", "triceps", "shoulders"],
    "1. Place your hands on the floor.\n2. Keep your body straight.\n3. Bend your elbows to lower your chest towards the ground.\n4. Then push back up"
)

display_exercise_details(push_up)



Perform as many Push-Ups for 30 seconds!
